Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 2.8228e-01, -3.8591e-02,  1.3227e-01, -3.1554e-01, -1.3401e-01,
          2.1180e-01],
        [-3.0956e-01,  2.9240e-01, -1.4708e-01,  1.7604e-01, -2.3346e-02,
         -2.0307e-01],
        [-3.6411e-01, -4.6149e-02,  3.8079e-02, -3.4818e-01, -2.6813e-01,
          3.5838e-01],
        [-2.5379e-01, -3.1458e-01,  3.4291e-01, -2.4858e-02,  3.6010e-01,
         -2.4865e-01],
        [-1.8683e-01, -3.9222e-01, -3.2887e-01,  1.0780e-01,  3.1693e-01,
         -2.2817e-01],
        [-3.0259e-01, -1.8737e-01,  3.4613e-02, -3.4274e-01, -2.7694e-02,
          1.7470e-01],
        [ 3.7399e-01, -2.7424e-01,  9.1091e-03,  7.0426e-02,  2.1839e-01,
         -3.1267e-01],
        [-1.3091e-01, -3.5432e-01, -1.4786e-01, -3.3149e-02,  1.3309e-01,
          1.1423e-01],
        [-3.1464e-01, -1.3486e-01,  3.2713e-01, -1.4798e-01,  2.9171e-01,
          2.7792e-01],
        [-2.7660e-01, -2.3076e-01, -1.8927e-01, -3.0893e-02,  4.5635e-03,
         -1.0785e-02],
        

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.0163],
        [0.0146],
        [0.0159],
        ...,
        [0.0154],
        [0.0176],
        [0.0162]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0786)
1 tensor(0.0755)
2 tensor(0.0725)
3 tensor(0.0697)
4 tensor(0.0669)
5 tensor(0.0644)
6 tensor(0.0620)
7 tensor(0.0598)
8 tensor(0.0577)
9 tensor(0.0558)
10 tensor(0.0541)
11 tensor(0.0524)
12 tensor(0.0509)
13 tensor(0.0496)
14 tensor(0.0483)
15 tensor(0.0472)
16 tensor(0.0464)
17 tensor(0.0456)
18 tensor(0.0449)
19 tensor(0.0443)
20 tensor(0.0437)
21 tensor(0.0431)
22 tensor(0.0426)
23 tensor(0.0421)
24 tensor(0.0418)
25 tensor(0.0414)
26 tensor(0.0412)
27 tensor(0.0410)
28 tensor(0.0408)
29 tensor(0.0406)
30 tensor(0.0405)
31 tensor(0.0404)
32 tensor(0.0402)
33 tensor(0.0400)
34 tensor(0.0398)
35 tensor(0.0396)
36 tensor(0.0393)
37 tensor(0.0390)
38 tensor(0.0388)
39 tensor(0.0385)
40 tensor(0.0381)
41 tensor(0.0378)
42 tensor(0.0375)
43 tensor(0.0373)
44 tensor(0.0370)
45 tensor(0.0367)
46 tensor(0.0364)
47 tensor(0.0361)
48 tensor(0.0358)
49 tensor(0.0355)
50 tensor(0.0352)
51 tensor(0.0348)
52 tensor(0.0345)
53 tensor(0.0342)
54 tensor(0.0339)
55 tensor(0.0336)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1453.76026702]
[1409.99999118] [1519.45628548]
[1128.99997044] [1181.51457524]
[12991.99967575] [11647.71707535]
[14720.0001564] [14460.25911331]
[2938.99994564] [2641.88813686]
[6635.00014687] [8001.16963768]
[11215.99973297] [9301.98052788]
[1040.99999189] [1440.49506736]
[5040.00003624] [5270.87294006]
[9541.99988747] [8570.25800323]
[3792.99988461] [5248.49931908]
[1002.0000155] [756.44350994]
[2542.00005341] [1515.41705561]
[6788.00001717] [6604.44801903]
[9386.00025749] [8372.74405861]
[1245.99996853] [1769.42247772]
[997.00000262] [226.3879272]
[708.00000262] [736.70446289]
[1435.99997544] [1434.29131961]
[2590.00006676] [2601.38724709]
[1392.99997497] [1689.95613194]
[11755.99946976] [9470.07455826]
[2007.99997377] [1236.1511848]
[2707.99998474] [2549.46775723]
[631.00000417] [663.60998809]
[652.99999881] [974.40492749]
[2492.00006247] [2654.64951801]
[594.99999416] [575.08841407]
[760.0000056] [1086.2700007]
[1179.00003028] [1064.91008019]
[1789.9999361] [1800

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [838.15519679]
[628.00000334] [685.93847764]
[442.00000328] [729.38800108]
[2284.99997044] [2588.08029222]
[1040.99999189] [818.13750613]
[809.99999654] [1228.22031522]
[3049.99987316] [3547.05431843]
[720.00000596] [666.82941043]
[877.99999249] [601.78011525]
[1292.00000429] [1636.3711195]
[7056.00004578] [4875.30292225]
[4176.99999142] [5928.90787315]
[13701.00020599] [15238.86817932]
[2948.0000515] [4089.58968449]
[16732.99985504] [14220.59253693]
[1815.00000048] [2018.24708557]
[16220.00016022] [11906.68750763]
[5967.00019073] [4695.83549213]
[473.00000042] [643.4057771]
[1078.9999795] [1027.36195827]
[452.00000319] [656.0454756]
[801.99998283] [776.69905365]
[2458.99994993] [2558.92482758]
[2492.99998236] [2518.98839092]
[582.00000203] [704.08407462]
[1405.9999671] [1456.16746116]
[8715.99991035] [11414.16798019]
[7634.00004578] [6471.93823814]
[506.99999839] [662.63025677]
[665.99999094] [746.93290198]
[1815.00000048] [1638.62920761]
[707.00001383] [657.76860106]
[

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1387.005395746521


## Conclusão



<p style = "text-align:justify">
    Como explicitado no início do notebook, a biblioteca pytorch sintetiza em si tudo o que aprendemos nos últimos sete experimentos. É claro que não invalida o trabalho e a atenção dado a eles, pois o aprendizado que tivemos por trás do funcionamento de redes neurais, mesmo que estas sejam as mais simples, foi e será fundamental para que possamos melhor trabalhar com problemas que requerem seu uso, mesmo que o façamos com a facilitude do Pytorch. Além disso, também aprendemos nesse notebook sobre a herança de classes, que complexifica ainda mais o conceito e funcionamento de classes. Basicamente, herança de classes nos permite definir uma classe a partir e métodos e propriedades de outras classes predefinidas. Desse jeito, podemos fazer nossa classe MLP se baseando totalmente na classe importada do pytorch, com todas as funções que precisamos.
</p>

## Playground

